In [1]:
%pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install selenium==4.4.3

^C
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.26.4 which is incompatible.


   ---------------------------------------- 0.0/986.0 kB ? eta -:--:--
   ------------------------------- ------- 786.4/986.0 kB 16.9 MB/s eta 0:00:01
   ---------------------------------------- 986.0/986.0 kB 3.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


We want to create a dataframe to store the comments and their sentiment.
- 0: negative
- 1: positive
- 2: neutral
Since we are labeling the sentiment by hand, we want to get the rating in each comment to simplifying the process of categorizing.

## IMDB Scraping
 Now I want to scrape the IMDB website for movies and their reviews. First, I will scrape some imdb URLs to collect the movie ids.

In [6]:
import requests
from bs4 import BeautifulSoup
import re
import json

def get_movie_id(list, url):
    # URL for the IMDb top 250 movies
    movie_id_list = list
    url = url
    # Set headers to simulate a browser visit
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }
    
    # Send an HTTP request to get the page content
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code != 200:
        print(f"Failed to retrieve page: {response.status_code}")
        return []
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text)
    

    # Find the title section of each movie which contains the link to the movie
    movies = soup.find_all("a", class_ = "ipc-title-link-wrapper")
    print(len(movies))
    # Collect the id of each movie by extracting it from the link
    
    for link in movies:
        href = link.get('href')  # Get the href attribute
        movie_id = href.split('/')[2]
        if not movie_id in movie_id_list:
            movie_id_list.append(movie_id)

    return movie_id_list


In [7]:
urls = [
    #best-rated action movies
    "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&countries=US&interests=in0000002&sort=user_rating,desc",

    #worst-rated action movies
    "https://www.imdb.com/search/title/?title_type=feature,tv_movie,tv_special,video&countries=US&interests=in0000002&sort=user_rating,asc",

    ]
#The chosen links are to ensure that the positive and negative comments are balanced
movie_id_list = []
for url in urls:
    get_movie_id(movie_id_list, url)

print(movie_id_list)

25
25
['tt0468569', 'tt1375666', 'tt0133093', 'tt0080684', 'tt6019206', 'tt0103064', 'tt0076759', 'tt0172495', 'tt15239678', 'tt1345836', 'tt0086190', 'tt1745960', 'tt0372784', 'tt1392190', 'tt1856101', 'tt0848228', 'tt0190332', 'tt1877830', 'tt5013056', 'tt3748528', 'tt2488496', 'tt0325710', 'tt9603212', 'tt4912910', 'tt0265086', 'tt0468547', 'tt15657202', 'tt0452668', 'tt2357423', 'tt26082308', 'tt4978420', 'tt3291150', 'tt1815862', 'tt1562871', 'tt5433138', 'tt0287978', 'tt23137904', 'tt8948208', 'tt14998742', 'tt9663764', 'tt14856980', 'tt8667828', 'tt6277462', 'tt5433140', 'tt2638144', 'tt1742334', 'tt13406136', 'tt0295701', 'tt3741700', 'tt1528100']


Next, I will visit each movie page with the retrieved ids and scrape the reviews and ratings.
To make sure the amount of comments for each category is balanced, I will collect them based on rating.

In [ ]:
def get_imdb_reviews(list,movie_id):
    # URL for the IMDb user reviews page
    urls = [#positive comments (rating 10)
        f"https://www.imdb.com/title/{movie_id}/reviews?ref_=tt_ql_3&rating=10", 
        #negative comments (rating 1)
        f"https://www.imdb.com/title/{movie_id}/reviews/?ref_=tt_ql_3&rating=1",
        #neutral comments (rating 5)
        f"https://www.imdb.com/title/{movie_id}/reviews/?ref_=tt_ql_3&rating=5" ]

    reviews_list = list
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36'
    }
    for url in urls:
        # Send an HTTP request to get the page content
        response = requests.get(url, headers = headers)
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find the review containers (limit to 5)
        review_containers = soup.find_all('div', class_='ipc-list-card__content', limit = 5) 

        for container in review_containers:
            review = container.find('div', class_='ipc-html-content-inner-div')
            rating = container.find('span', class_='ipc-rating-star')
            rating_score =  int(rating.text.split("/")[0])
            if review:
                obj = {"comment": review.text,
                    "sentiment": 1 if rating_score == 10 else 0 if rating_score ==1 else 2}
                
                reviews_list.append(obj)
  
    return reviews_list


In [ ]:
# Collect the text of each review
reviews_list = []
for movie_id in movie_id_list:
    get_imdb_reviews(reviews_list,movie_id)

print(len(reviews_list))

In [11]:
import pandas as pd

In [19]:
#add the data to the dataframe
df = pd.DataFrame(reviews_list)

In [20]:
df.head()

,comment,sentiment
0,Best movie ever. Heath ledger's work is phenom...,1
1,This movie is a work of art. The finest sequel...,1
2,It is just what you want for the best movie. G...,1
3,We've been subjected to enormous amounts of hy...,1
4,Let me start off by saying that I hated this f...,2


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493 entries, 0 to 492
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   comment    493 non-null    object
 1   sentiment  493 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 7.8+ KB


In [26]:
# Get the frequency of each unique value in the 'sentiment' column
value_counts = df['sentiment'].value_counts()

# Display the result
print(value_counts)

sentiment
1    196
2    152
0    145
Name: count, dtype: int64


In [27]:
# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

try:
    df.to_csv("movie_reviews.csv", index=False)  # index=False if you don’t need row indices
    print("File saved successfully.")
except Exception as e:
    print("Error:", e)


File saved successfully.
